# Método dos Elementos Finitos - Trabalho 3

Universidade Federal Fluminense

Disciplina ministrada pelo Prof. Marco Ferro

<marcoferro@id.uff.br>

Aluno Noé de Lima

<noe_lima@id.uff.br>

Este trabalho visa aplicar o MEF a uma estrutura de vigas.

Primeiro semestre de 2020

\vfill

A célula a seguir configura o Jupyter-Notebook para exibir as equações matemáticas no formato do ambiente $\LaTeX$ e importa as bibliotecas necessárias.

In [1]:
import numpy as np
import sympy as sym
import plotly.graph_objects as go
import bokeh.plotting as bk
import json
from plotly.offline import init_notebook_mode, iplot
from bokeh.io import output_notebook, export_png
from bokeh.models import ColumnDataSource, HoverTool
from IPython.display import Image
init_notebook_mode(connected=True)
output_notebook()
sym.init_printing(use_latex='mathjax',latex_mode='equation*')
!uname -a

Loading BokehJS ...

Linux DESKTOP-CR7O8A2 4.19.104-microsoft-standard #1 SMP Wed Feb 19 06:37:35 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux


\cleardoublepage

\tableofcontents

\cleardoublepage

# Introdução

Em uma estrutura de treliças, cada barra é um elemento, e o Método dos Elementos Finitos coincide com a Análise Matricial de Estruturas. Considerando que todos os esforços na treliça são normais, não há cargas transversais, o problema, visto do ponto de vista das barras, é unidimensional, pois, da mesma forma que os esforços, os deslocamentos também são na mesma direção das barras.

No caso de vigas, havendo momento fletor (ou torçor), tanto direto como em consequência de esforço cortante, há deformação no sentido perpendicular à viga, bem como rotação pontual em decorrência da deformação. Assim, uma viga será discretizada na forma de uma malha compostas por pequenos segmentos.

Em um Elemento de Viga, cujo eixo principal é o eixo $x$, temos:

* $V_{1} \rightarrow$ Força perpendicular no início do Elemento;
* $V_{2} \rightarrow$ Força perpendicular no final do Elemento;
* $P \rightarrow$ Carga perpendicular distribuída;
* $M_{1} \rightarrow$ Momento no início do Elemento;
* $M_{2} \rightarrow$ Momento no final do Elemento;
* $L \rightarrow$ Comprimento do Elemento.

Bem como os deslocamentos:
* $u_{1}=\delta_{1}\rightarrow$ deflexão $\delta$ no início do Elemento;
* $u_{2}=\theta_{1}\rightarrow$ inclinação $\theta$ no início do Elemento;
* $u_{3}=\delta_{2}\rightarrow$ deflexão $\delta$ no final do Elemento;
* $u_{4}=\theta_{2}\rightarrow$ inclinação $\theta$ no final do Elemento.

No caso de uma viga retangular de largura $b$ e altura $h$, temos o momento de inércia:

\begin{equation*}
    I = \frac{bh^{3}}{12}
\end{equation*}

Assim, sendo o Módulo de Elasticidade $E$ constante, o produto $EI$ também será.

Temos, portanto, a seguinte equação diferencial da viga:

\begin{equation*}
    EI\frac{d^{4}u}{dx^{4}} = P
\end{equation*}

Tem-se que:

\begin{equation*}
    V = EI\frac{d^{3}u}{dx^{3}}
\end{equation*}

E

\begin{equation*}
    M = EI\frac{d^{2}u}{dx^{2}}
\end{equation*}

Aplicando a SRP - Sentença de Resíduos Ponderados - na EDO da viga, resulta:

\begin{equation*}
    \int_{\Omega}{N_{i}R}d\Omega = 0
\end{equation*}

$$\therefore$$

\begin{equation*}
    \int_{0}^{L}{N_{i}\left(EI\frac{d^{4}u}{dx^{4}}-P\right)}dx = 0
\end{equation*}


In [2]:
x = sym.symbols('x')
P = sym.symbols('P')
E,I,L = sym.symbols('E,I,L')
u = sym.Function('u')
N_i = sym.symbols('N_i',cls=sym.Function)
R = E*I*sym.diff(u(x),x,4)-P
SRP = sym.integrate(N_i(x)*R,(x,0,L))
display(SRP.doit())

L                             
⌠                             
⎮ ⎛      4          ⎞         
⎮ ⎜     d           ⎟         
⎮ ⎜E⋅I⋅───(u(x)) - P⎟⋅Nᵢ(x) dx
⎮ ⎜      4          ⎟         
⎮ ⎝    dx           ⎠         
⌡                             
0                             

Essa integral não pode ser resolvida pelos métodos sym.dsolve(sym.Eq(SRP,0),u(x)) ou SRP.doit(), pois o nível de complexidade ainda não foi implementado na biblioteca.

Utilizando a integração por partes, obtém-se o seguinte:

In [3]:
eq = sym.integrate(E*I*(sym.diff(N_i(x),x)*sym.diff(u(x),x,3)),(x,0,L))
display(eq.doit())

    L                       
    ⌠                       
    ⎮             3         
    ⎮ d          d          
E⋅I⋅⎮ ──(Nᵢ(x))⋅───(u(x)) dx
    ⎮ dx          3         
    ⎮           dx          
    ⌡                       
    0                       

# Funções de Hermite

In [4]:
N_1 = 1/2 - (3*(x-L/2))/(2*L) + (2*(x-L/2)**3)/(L**3)

N_2 = L/8 - (x-L/2)/4 - ((x-L/2)**2)/(2*L) + ((x-L/2)**3)/(L**2)

N_3 = 1/2 + (3*(x-L/2))/(2*L) - (2*(x-L/2)**3)/(L**3)

N_4 = -L/8 - (x-L/2)/4 + ((x-L/2)**2)/(2*L) + ((x-L/2)**3)/(L**2)

display(N_1)
display(N_2)
display(N_3)
display(N_4)

                               3
        3⋅L           ⎛  L    ⎞ 
      - ─── + 3⋅x   2⋅⎜- ─ + x⎟ 
         2            ⎝  2    ⎠ 
0.5 - ─────────── + ────────────
          2⋅L             3     
                         L      

                 2            3
        ⎛  L    ⎞    ⎛  L    ⎞ 
        ⎜- ─ + x⎟    ⎜- ─ + x⎟ 
L   x   ⎝  2    ⎠    ⎝  2    ⎠ 
─ - ─ - ────────── + ──────────
4   4      2⋅L            2    
                         L     

                               3
        3⋅L           ⎛  L    ⎞ 
      - ─── + 3⋅x   2⋅⎜- ─ + x⎟ 
         2            ⎝  2    ⎠ 
0.5 + ─────────── - ────────────
          2⋅L             3     
                         L      

               2            3
      ⎛  L    ⎞    ⎛  L    ⎞ 
      ⎜- ─ + x⎟    ⎜- ─ + x⎟ 
  x   ⎝  2    ⎠    ⎝  2    ⎠ 
- ─ + ────────── + ──────────
  4      2⋅L            2    
                       L     

# Matriz de Rigidez Local

\begin{equation*}
    K_{i,j} = \int_{0}^{L}{\frac{d^{2}N_{i}\left(x\right)}{dx^{2}}\frac{d^{2}N_{j}\left(x\right)}{dx^{2}}}dx
\end{equation*}

In [5]:
K_e = sym.Matrix([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])

K_e[0,0] = E*I*sym.integrate(sym.diff(N_1,x,2)*sym.diff(N_1,x,2),[x,0,L])
K_e[0,1] = E*I*sym.integrate(sym.diff(N_1,x,2)*sym.diff(N_2,x,2),[x,0,L])
K_e[0,2] = E*I*sym.integrate(sym.diff(N_1,x,2)*sym.diff(N_3,x,2),[x,0,L])
K_e[0,3] = E*I*sym.integrate(sym.diff(N_1,x,2)*sym.diff(N_4,x,2),[x,0,L])

K_e[1,0] = K_e[0,1]
K_e[1,1] = E*I*sym.integrate(sym.diff(N_2,x,2)*sym.diff(N_2,x,2),[x,0,L])
K_e[1,2] = E*I*sym.integrate(sym.diff(N_2,x,2)*sym.diff(N_3,x,2),[x,0,L])
K_e[1,3] = E*I*sym.integrate(sym.diff(N_2,x,2)*sym.diff(N_4,x,2),[x,0,L])

K_e[2,0] = K_e[0,2]
K_e[2,1] = K_e[1,2]
K_e[2,2] = E*I*sym.integrate(sym.diff(N_3,x,2)*sym.diff(N_3,x,2),[x,0,L])
K_e[2,3] = E*I*sym.integrate(sym.diff(N_3,x,2)*sym.diff(N_4,x,2),[x,0,L])

K_e[3,0] = K_e[0,3]
K_e[3,1] = K_e[1,3]
K_e[3,2] = K_e[2,3]
K_e[3,3] = E*I*sym.integrate(sym.diff(N_4,x,2)*sym.diff(N_4,x,2),[x,0,L])

display(K_e/(E*I/L))

⎡ 12    6   -12    6 ⎤
⎢ ──    ─   ────   ─ ⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     4   ───    2 ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢-12   -6    12   -6 ⎥
⎢────  ───   ──   ───⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     2   ───    4 ⎥
⎣ L          L       ⎦

# Vetor Local de Forças Nodais

In [6]:
V_1,M_1,V_2,M_2 = sym.symbols('V_1,M_1,V_2,M_2')
f_e = sym.Matrix([V_1,-M_1,-V_2,M_2])

f_e[0] += P*L/2
f_e[1] += P*L**2/12
f_e[2] += P*L/2
f_e[3] += -P*L**2/12

display(f_e)

⎡ L⋅P       ⎤
⎢ ─── + V₁  ⎥
⎢  2        ⎥
⎢           ⎥
⎢  2        ⎥
⎢ L ⋅P      ⎥
⎢ ──── - M₁ ⎥
⎢  12       ⎥
⎢           ⎥
⎢ L⋅P       ⎥
⎢ ─── - V₂  ⎥
⎢  2        ⎥
⎢           ⎥
⎢   2       ⎥
⎢  L ⋅P     ⎥
⎢- ──── + M₂⎥
⎣   12      ⎦

# Exemplo Analítico

A partir da definição da Matriz de Rigidez e do Vetor de Cargas Nodais dentro de um único Elemento $e$, podemos expandir para uma viga (ou diversas vigas) em uma estrutura dividida em $n$ Elementos.

Vamos utilizar, como exemplo, uma viga biapoiada de $2~m$ de comprimento, sendo o apoio no início do vão, de segundo gênero, e o do final do vão, de primeiro gênero. A viga está sujeita a uma carga uniformemente distribuída $P = 10~kN/m$ e tem o produto constante $EI = 10^{2}kNm^{2}$. O problema consiste em calcular o deslocamento e o momento no centro do vão, bem como as reações e as rotações nos apoios.

A solução analítica do problema é:

$$u =$$

In [7]:
u = (5/384)*(P*L**4)/(E*I)
display(u)
display(u.subs(P,10).subs(L,2).subs(E*I,10**2))

                    4  
0.0130208333333333⋅L ⋅P
───────────────────────
          E⋅I          

0.0208333333333333

$$M =$$

In [8]:
M = (P*L**2)/8
display(M)
display(M.subs(P,10).subs(L,2))

 2  
L ⋅P
────
 8  

5

$$\theta_{A} = -\theta_{B} =$$

In [9]:
theta_A = (P*L**3)/(24*E*I)
display(theta_A)
display(theta_A.subs(P,10).subs(L,2).subs(E*I,10**2))

  3   
 L ⋅P 
──────
24⋅E⋅I

1/30

$$V_{A} = V_{B} =$$

In [10]:
V_A = P*L/2
display(V_A)
display(V_A.subs(P,10).subs(L,2))

L⋅P
───
 2 

10

Considerando dois Elementos na solução, temos então:

## Elemento 1

### Variáveis de Deformação

* $u_{1}^{1}\rightarrow$ deflexão $\delta$ no primeiro apoio;

* $u_{2}^{1}\rightarrow$ rotação $\theta$ no primeiro apoio;

* $u_{3}^{1}\rightarrow$ deflexão $\delta$ no centro do vão;

* $u_{4}^{1}\rightarrow$ rotação $\theta$ no centro do vão.

### Variáveis de Carregamento

* $M_{1}^{1}\rightarrow$ Momento Fletor no primeiro apoio;

* $V_{1}^{1}\rightarrow$ Força Cortante no primeiro apoio;

* $M_{2}^{1}\rightarrow$ Momento Fletor no centro do vão;

* $V_{2}^{1}\rightarrow$ Força Cortante no centro do vão.

## Elemento 2

### Variáveis de Deformação

* $u_{1}^{2}\rightarrow$ deflexão $\delta$ no centro do vão;

* $u_{2}^{2}\rightarrow$ rotação $\theta$ no centro do vão;

* $u_{3}^{2}\rightarrow$ deflexão $\delta$ no segundo apoio;

* $u_{4}^{2}\rightarrow$ rotação $\theta$ no segundo apoio.

### Variáveis de Carregamento

* $M_{1}^{2}\rightarrow$ Momento Fletor no centro do vão;

* $V_{1}^{2}\rightarrow$ Força Cortante no centro do vão;

* $M_{2}^{2}\rightarrow$ Momento Fletor no segundo apoio;

* $V_{2}^{2}\rightarrow$ Força Cortante no csegundo apoio.

### Nó Central

 No nó central há a conexão entre o segundo nó do primeiro Elemento e o primeiro nó do segundo Elemento. Portanto, seus valores coincidem. Assim,

\begin{equation*}
    M_{2}^{1} = M_{1}^{2}
\end{equation*}

E

\begin{equation*}
    V_{1}^{2} = V_{2}^{1}
\end{equation*}

## Cálculo das Matrizes de Rigidez Locais

$$\mathbf{K}_{e} = \frac{EI}{L}\times$$

In [11]:
display(K_e/(E*I/L))

⎡ 12    6   -12    6 ⎤
⎢ ──    ─   ────   ─ ⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     4   ───    2 ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢-12   -6    12   -6 ⎥
⎢────  ───   ──   ───⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     2   ───    4 ⎥
⎣ L          L       ⎦

## Elemento 1

$$K_{1} = 10^{2}\times$$

In [12]:
K_1 = K_e.subs(E*I,10**2).subs(L,1)
display(K_1/100)

⎡12   6   -12  6 ⎤
⎢                ⎥
⎢ 6   4   -6   2 ⎥
⎢                ⎥
⎢-12  -6  12   -6⎥
⎢                ⎥
⎣ 6   2   -6   4 ⎦

## Elemento 2

$$K_{2} = 10^{2}\times$$

In [13]:
K_2 = K_e.subs(E*I,10**2).subs(L,1)
display(K_1/100)

⎡12   6   -12  6 ⎤
⎢                ⎥
⎢ 6   4   -6   2 ⎥
⎢                ⎥
⎢-12  -6  12   -6⎥
⎢                ⎥
⎣ 6   2   -6   4 ⎦

## Cálculo do Vetor $\vec{f}$

In [14]:
display(f_e)

⎡ L⋅P       ⎤
⎢ ─── + V₁  ⎥
⎢  2        ⎥
⎢           ⎥
⎢  2        ⎥
⎢ L ⋅P      ⎥
⎢ ──── - M₁ ⎥
⎢  12       ⎥
⎢           ⎥
⎢ L⋅P       ⎥
⎢ ─── - V₂  ⎥
⎢  2        ⎥
⎢           ⎥
⎢   2       ⎥
⎢  L ⋅P     ⎥
⎢- ──── + M₂⎥
⎣   12      ⎦

### Elemento 1

In [15]:
V_1_1,M_1_1,V_2_1,M_2_1 = sym.symbols('V_1_1,M_1_1,V_2_1,M_2_1')
f_1 = f_e.subs(L,1).subs(P,10).subs(V_1,V_1_1).subs(V_2,V_2_1).subs(M_1,M_1_1).subs(M_2,M_2_1)
display(f_1)

⎡ V₁ ₁ + 5 ⎤
⎢          ⎥
⎢5/6 - M₁ ₁⎥
⎢          ⎥
⎢ 5 - V₂ ₁ ⎥
⎢          ⎥
⎣M₂ ₁ - 5/6⎦

### Elemento 2

In [16]:
V_1_2,M_1_2,V_2_2,M_2_2 = sym.symbols('V_1_2,M_1_2,V_2_2,M_2_2')
f_2 = f_e.subs(L,1).subs(P,10).subs(V_1,V_1_2).subs(V_2,V_2_2).subs(M_1,M_1_2).subs(M_2,M_2_2)
display(f_2)

⎡ V₁ ₂ + 5 ⎤
⎢          ⎥
⎢5/6 - M₁ ₂⎥
⎢          ⎥
⎢ 5 - V₂ ₂ ⎥
⎢          ⎥
⎣M₂ ₂ - 5/6⎦

### Vetor de Forças Global

In [17]:
f = sym.zeros(6,1)
f[0:4,0] += f_1[0:4,0]
f[2:6,0] += f_2[0:4,0]
display(f)

⎡    V₁ ₁ + 5    ⎤
⎢                ⎥
⎢   5/6 - M₁ ₁   ⎥
⎢                ⎥
⎢V₁ ₂ - V₂ ₁ + 10⎥
⎢                ⎥
⎢  -M₁ ₂ + M₂ ₁  ⎥
⎢                ⎥
⎢    5 - V₂ ₂    ⎥
⎢                ⎥
⎣   M₂ ₂ - 5/6   ⎦

Entretanto, lambramos que no nó central, temos:

\begin{equation*}
    \left[\begin{matrix}
        -V_{2}^{1} & + & V_{1}^{2} & = & 0 \\
        M_{2}^{1} & - & M_{1}^{2} & = & 0
    \end{matrix}\right.
\end{equation*}

Portanto, $\vec{f}$ torna-se:

In [18]:
f_G = f.subs(-V_2_1+V_1_2,0).subs(M_2_1-M_1_2,0)
display(f_G)

⎡ V₁ ₁ + 5 ⎤
⎢          ⎥
⎢5/6 - M₁ ₁⎥
⎢          ⎥
⎢    10    ⎥
⎢          ⎥
⎢    0     ⎥
⎢          ⎥
⎢ 5 - V₂ ₂ ⎥
⎢          ⎥
⎣M₂ ₂ - 5/6⎦

## Matriz de Rigidez Global

A Matriz de Rigidez Global fica:

$$K_G = 10^{2}\times$$

In [19]:
K_G = sym.zeros(6,6)
K_G[0:4,0:4] += K_1
K_G[2:6,2:6] += K_2
display(K_G/100)

⎡12   6   -12  6    0   0 ⎤
⎢                         ⎥
⎢ 6   4   -6   2    0   0 ⎥
⎢                         ⎥
⎢-12  -6  24   0   -12  6 ⎥
⎢                         ⎥
⎢ 6   2    0   8   -6   2 ⎥
⎢                         ⎥
⎢ 0   0   -12  -6  12   -6⎥
⎢                         ⎥
⎣ 0   0    6   2   -6   4 ⎦

## Vetor de Deslocamentos

Considerando as condições de contorno, temos:

\begin{equation*}
    \left\{\begin{matrix}
        u_{1}^{1} = u_1 & = & 0 \\
        u_{1}^{2} = u_5 & = & 0 \\
        M_{1}^{1} & = & 0 \\
        M_{2}^{2} & = & 0 \\
    \end{matrix}\right.
\end{equation*}

Portanto,

In [20]:
u_1,u_2,u_3,u_4,u_5,u_6 = sym.symbols('u_1,u_2,u_3,u_4,u_5,u_6')
u = sym.Matrix([u_1,u_2,u_3,u_4,u_5,u_6]).subs(u_1,0).subs(u_5,0)
f_G = f_G.subs(M_1_1,0).subs(M_2_2,0)
display(u)
display(f_G)

⎡0 ⎤
⎢  ⎥
⎢u₂⎥
⎢  ⎥
⎢u₃⎥
⎢  ⎥
⎢u₄⎥
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣u₆⎦

⎡V₁ ₁ + 5⎤
⎢        ⎥
⎢  5/6   ⎥
⎢        ⎥
⎢   10   ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢5 - V₂ ₂⎥
⎢        ⎥
⎣  -5/6  ⎦

Assim, o sistema $\mathbf{K}\cdot\vec{u}=\vec{f}$ torna-se:

In [21]:
eq = sym.Eq(K_G * u, f_G)
display(eq)

⎡600⋅u₂ - 1200⋅u₃ + 600⋅u₄ ⎤   ⎡V₁ ₁ + 5⎤
⎢                          ⎥   ⎢        ⎥
⎢ 400⋅u₂ - 600⋅u₃ + 200⋅u₄ ⎥   ⎢  5/6   ⎥
⎢                          ⎥   ⎢        ⎥
⎢-600⋅u₂ + 2400⋅u₃ + 600⋅u₆⎥   ⎢   10   ⎥
⎢                          ⎥ = ⎢        ⎥
⎢ 200⋅u₂ + 800⋅u₄ + 200⋅u₆ ⎥   ⎢   0    ⎥
⎢                          ⎥   ⎢        ⎥
⎢-1200⋅u₃ - 600⋅u₄ - 600⋅u₆⎥   ⎢5 - V₂ ₂⎥
⎢                          ⎥   ⎢        ⎥
⎣ 600⋅u₃ + 200⋅u₄ + 400⋅u₆ ⎦   ⎣  -5/6  ⎦

A partir deste sistema, considerando as condições de contorno, podemos obter o seguinte sistema reduzido:

In [22]:
f_R = f_G.copy()
f_R -= u[0,0]*K_G[:,0]
f_R -= u[4,0]*K_G[:,4]

K_R = K_G.copy()
K_R.row_del(4)
K_R.col_del(4)
K_R.row_del(0)
K_R.col_del(0)
f_R.row_del(4)
f_R.row_del(0)

display(K_R)
display(f_R)

⎡400   -600  200   0 ⎤
⎢                    ⎥
⎢-600  2400   0   600⎥
⎢                    ⎥
⎢200    0    800  200⎥
⎢                    ⎥
⎣ 0    600   200  400⎦

⎡5/6 ⎤
⎢    ⎥
⎢ 10 ⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎣-5/6⎦

A solução deste sistema fornece:

$$u' = 10^{-2}\times$$

In [23]:
u_R = K_R.solve(f_R)
display(u_R*100)

⎡10/3 ⎤
⎢     ⎥
⎢ 25  ⎥
⎢ ──  ⎥
⎢ 12  ⎥
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎣-10/3⎦

Temos, portanto,

\begin{equation*}
    \left\{\begin{matrix}
        u_{2}^{1} = u_{2} & = & \frac{10}{3}\times10^{-2} rad \\
        u_{3}^{1} = u_{1}^{2} = u_{3} & = & \frac{25}{12}\times10^{-2} m \\
        u_{4}^{1} = u_{2}^{2} = u_{4} & = & 0 \\
        u_{4}^{2} = u_{6} & = & -\frac{10}{3}\times10^{-2} rad \\
    \end{matrix}\right.
\end{equation*}

Temos, agora, o seguinte vetor de deslocamentos:

$$u = 10^{-2}$$

In [24]:
u = u.subs(u_2,1/30).subs(u_3,25/1200).subs(u_4,0).subs(u_6,-1/30)
display(100*u)

⎡        0        ⎤
⎢                 ⎥
⎢3.33333333333333 ⎥
⎢                 ⎥
⎢2.08333333333333 ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎣-3.33333333333333⎦

Agora podemos calcular as forças nodais.

$$\vec{f} = $$

In [25]:
f = K_G*u
display(f,f_G)

⎡       -5.0       ⎤
⎢                  ⎥
⎢0.833333333333334 ⎥
⎢                  ⎥
⎢       10.0       ⎥
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢       -5.0       ⎥
⎢                  ⎥
⎣-0.833333333333334⎦

⎡V₁ ₁ + 5⎤
⎢        ⎥
⎢  5/6   ⎥
⎢        ⎥
⎢   10   ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢5 - V₂ ₂⎥
⎢        ⎥
⎣  -5/6  ⎦

Temos, portanto,

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{1} + 5 & = & -5 \\
        5 - V_{2}^{2} & = & -5
    \end{matrix}\right.
\end{equation*}

Logo,

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{1} & = & -10 \\
        V_{2}^{2} & = & 10 \\
    \end{matrix}\right.
\end{equation*}

Temos, portanto, os seguintes valores de $\vec{u}_{e}^{1}$ no Elemento 1:

$$u_e^{1} =$$

In [26]:
u_e_1 = u[0:4,0]
display(u_e_1)

⎡        0         ⎤
⎢                  ⎥
⎢0.0333333333333333⎥
⎢                  ⎥
⎢0.0208333333333333⎥
⎢                  ⎥
⎣        0         ⎦

Da mesma forma, temos como calcular o vetor de Forças Nodais $\vec{f}_{e}^{1}$.

$$\vec{f}_{e}^{1} =$$

In [27]:
f_e_1 = K_1*u_e_1
display(f_e_1,f_e.subs(L,1).subs(P,10))

⎡      -5.0       ⎤
⎢                 ⎥
⎢0.833333333333334⎥
⎢                 ⎥
⎢       5.0       ⎥
⎢                 ⎥
⎣-5.83333333333333⎦

⎡ V₁ + 5 ⎤
⎢        ⎥
⎢5/6 - M₁⎥
⎢        ⎥
⎢ 5 - V₂ ⎥
⎢        ⎥
⎣M₂ - 5/6⎦

De onde resulta:

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{1} & = & -10~kN \\
        M_{1}^{1} & = & 0 \\
        V_{2}^{1} & = & 0 \\
        M_{2}^{1} & = & -5~kN\cdot m
    \end{matrix}\right.
\end{equation*}

Seguindo o mesmo raciocínio, podemos calcular o vetor de Forças Nodais do Elemento 2, $\vec{f}_{e}^{2}$.

$$\vec{f}_{e}^{2} =$$

In [28]:
u_e_2 = u[2:6,0]
f_e_2 = K_2*u_e_2
display(f_e_2,f_e.subs(L,1).subs(P,10))

⎡       5.0        ⎤
⎢                  ⎥
⎢ 5.83333333333333 ⎥
⎢                  ⎥
⎢       -5.0       ⎥
⎢                  ⎥
⎣-0.833333333333334⎦

⎡ V₁ + 5 ⎤
⎢        ⎥
⎢5/6 - M₁⎥
⎢        ⎥
⎢ 5 - V₂ ⎥
⎢        ⎥
⎣M₂ - 5/6⎦

De onde resulta:

\begin{equation*}
    \left\{\begin{matrix}
        V_{1}^{2} & = & 0 \\
        M_{1}^{2} & = & -5~kN\cdot m \\
        V_{2}^{2} & = & 10~kN \\
        M_{2}^{2} & = & 0
    \end{matrix}\right.
\end{equation*}

# Caso Geral

# Solução Por Código

A seguir, vamos elaborar um código em Python para solucionar uma viga a partir de um arquivo de entrada no formato JSON cotendo as informações da estrutura.

vamos começar definindo uma classe para conter os nós da estrutura.

## Classe "Nó"

A classe que conterá os nós é genérica e tem capacedade de armazenar informações sobre um sistema tridimensional, contemplando, assim, a localização espacial do nó em coordenadas $(x,y,z)$, bem como informações sobre as restrições (de translação e rotação) impostas por apoios, bem como as cargas pontuais aplicadas sobre este, sendo forças ou momentos.

Embora esta classe seja de carater genérico, apenas os campos aplicáveis ao caso da viga unidimensional serão efetivamente utilizados.

In [29]:
# Class to store and update nodes and nodes data as locate, if have constrains and loads, as well calculated reactions and deslocs
class node:
    def __init__(self, i, P=np.array([0.0,0.0,0.0]), S=[False,False,False,False,False,False], L=np.array([0.0,0.0,0.0,0.0,0.0,0.0]), tag=''):
            self.index = i                          # node index reference
            self.locate = P                         # (x,y,z)
            self.support = S                        # (Rx,Ry,Rz,Rmx,Rmy,Rmz) bool if exists
            self.load = L                           # (Fx,Fy,Fz,Mx,My,Mz)
            self.tag = tag                          # String name
            self.reaction = np.array([0.0,0.0,0.0,0.0,0.0,0.0]) # (Rx,Ry,Rz,Rmx,Rmy,Rmz) to calculate
            self.u = np.array([0.0,0.0,0.0,0.0,0.0,0.0])        # (dx,dy,dz,theta_x,theta_y,theta_z) to calculate
    
    def Reaction(self,R=np.array([0.0,0.0,0.0,0.0,0.0,0.0])):
        self.reaction = R                           # Calculated values

    def Desloc(self,u=np.array([0.0,0.0,0.0,0.0,0.0,0.0])):
        self.u = u                                  # Calculated values

## Classe "Viga"

Da mesma forma, precisamos de uma classe para operar com as barras que conectam os nós, assim como as propriedades de carga e deformação.

A classe abaixo lida com barras como elementos finitos locais. As barras reais serão criadas como um elemento único e subdivididas em elementos com nós intermediários.

Por simplificação, serão consideradas vigas unidimensionais ao longo do eixo $x$, com esforços cortantes na direção $y$ e momento fletor em $z$. A classe Viga é, na prática, uma sucessão de pontos em $x$, onde cada segmento vai do ponto $x_{i}$ para o ponto $x_{i+1}$, sendo $y=0$ e $z=0$. Como estamos considerando os esforços cortantes e de momento fletor, os deslocamentos ocorrerão no eixo $y$.

A implementação do código da classe "Viga" abaixo está configurada para uma viga tridimensional. com esforços diversos nas estremidades, bem como esforços distribuídos na forma $a+bx+cx^{2}$, mas a matriz de rigidez está configurada para esforços contantes, momento fletor e carga distribuída constante apenas ($a=P, b=0, c=0$). Assim, um segmento de viga é dividido em $n$ elementos finitos e retorna a matriz de rigidez e vetor de forças nodais para estes esforços apenas.

In [30]:
# Class to store and calculate bars or elements
class element:
    def __init__(self,i,first,last,E,G,S,I,J,P=np.array([0,0,0]),tag='',n=10):
        self.index = i      # Index
        self.n = n          # Grid size
        self.tag = tag      # Element name
        self.start = first  # Start node
        self.end = last     # End node
        self.grid = []      # Fake nodes from grid
        self.E = E          # Elasticity constant
        self.G = G          # Transversal Elasticity constant
        self.A = S          # Stretch
        self.I = I          # Inertia constant
        self.J = J          # Polar inertia constant
        self.load = P       # Loads with a+bx+cx**2 constants in array (a,b,c)
        self.set_grid(n)

    def set_grid(self,n):
        self.n = n
        self.grid = []
        ds = (self.end.locate - self.start.locate)/n
        for i in range(1,n):
            loc = self.start.locate+ds*i
            self.grid.append(node(0,P=loc))

    """
    To K matrix, we are considerating the local variables below:
        * u_0 -> dx_1
        * u_1 -> dy_1
        * u_2 -> dz_1
        * u_3 -> theta_x_1 (yz)
        * u_4 -> theta_y_1 (zx)
        * u_5 -> theta_z_1 (xy)
        * u_6 -> dx_2
        * u_7 -> dy_2
        * u_8 -> dz_2
        * u_9 -> theta_x_2 (yz)
        * u_10 -> theta_y_2 (zx)
        * u_11 -> theta_z_2 (xy)
    For two-dimensional plan just use u_0, u_1, u_3, u_6, u_7, u_11
    """

    def K_2d(self):
        A = self.A
        E = self.E
        G = self.G
        I = self.I
        J = self.J
        L = np.linalg.norm(self.end.locate - self.start.locate) / self.n
        K = np.zeros([2*(self.n+1),2*(self.n+1)])
        for k in range(self.n):
            K_l = (E*I/L) * np.array([
                [12/L**2,   6/L,    -12/L**2,   6/L ],
                [6/L,       4,      -6/L,       2   ],
                [-12/L**2,  -6/L,   12/L**2,    -6/L],
                [6/L,       2,      -6/L,       4   ]
            ])
            K[2*k:2*k+4,2*k:2*k+4] += K_l
        return K

    def f_2d(self):
        P = self.load[0]
        L = np.linalg.norm(self.end.locate - self.start.locate) / self.n
        f = np.zeros(2*(self.n+1))
        for k in range(self.n):
            if k == 0:
                V_1 = self.start.load[1]
                M_1 = self.start.load[5]
                V_2 = 0
                M_2 = 0
            elif k == self.n-1:
                V_1 = 0
                M_1 = 0
                V_2 = 0
                M_2 = 0
            else:
                V_1 = 0
                M_1 = 0
                V_2 = self.end.load[1]
                M_2 = self.end.load[5]
            f_l = np.array([
                L*P/2 + V_1,
                (L**2)*P/12 - M_1,
                L*P/2 - V_2,
                -(L**2)*P/12 + M_2
            ])
            f[2*k:2*k+4] += f_l
        if self.start.support[1]:
            f[0] = np.NaN
        if self.start.support[5]:
            f[1] = np.NaN
        if self.end.support[1]:
            f[-2] = np.NaN
        if self.end.support[5]:
            f[-1] = np.NaN
        return f

## Entrada de Arquivo JSON

Entre as funções auxiliares, também utilizaremos uma função para ler um arquivo JSON e armazenar em uma variável na forma de dicionário Python. Alternativamente, esta função poderia estar contida na própria classe da estrutura que será criada mais adiante, de forma que o caminho seria passado como parâmetro, entretanto, a criação separada permite visualizar o conteúdo do arquivo fora da classe.

In [31]:
def readjson(path):
    file = None
    try:
        with open(path,'r') as f:
            file = json.load(f)
    except IOError as err:
        print('File Error: ' + str(err))
    except JSONDecodeError as err:
        print('JSON Error: ' + str(err))
    finally:
        return file

## Classe "Estrutura"

Por fim, utilizaremos uma classe para processar toda a estrutura. Esta classe conterá listas com todos os nós e barras da estrutura, bem como os métodos necessários para calcular os deslocamentos dos nós, esforços e deformações nas barras.

A chamada para a criação de um objeto de estrutura será pelo fornecimento de um dicionário lido através de um arquivo JSON contendo todos os dados da estrutura, como os nós, apoios, constantes, ligações entre os nós e carregamentos.

Conforme descrito na classe "Viga", a estrutura será uma sucessão de segmentos de viga, formando uma única viga, com nós nas extremidades e nos pontos de aplicação de carga, ou de início e fim de carga distribuída constante. Portanto, apesar de ler e plotar quaisquer estruturas, bidimensionais ou tridimensionais, apenas será capaz de resolver, pelo métodos dos deslocamentos, estruturas reticuladas contínuas horizontais com cargas verticais e momentos em $z$.

In [32]:
# Structure data class
class Structure:
    def __init__(self,data,grid=10):
        self.tag = data["tag"]
        self.n = data["n"]
        self.dimension = data["dim"]
        self.grid = grid
        self._solved = False
        self.nodes = []
        self.bars = []
        for name, value in data['nodes'].items():
            P = np.array([value["x"],value["y"],value["z"]])
            S=[False,False,False,False,False,False]
            L=np.array([value["Fx"], value["Fy"], value["Fz"], value["Mx"], value["My"], value["Mz"]])
            if value["Tx"]:
                S[0] = True
            if value["Ty"]:
                S[1] = True
            if value["Tz"]:
                S[2] = True
            if value["Rx"]:
                S[3] = True
            if value["Ry"]:
                S[4] = True
            if value["Rz"]:
                S[5] = True
            newnode = node(value["id"],P,S,L,name)
            self.nodes.append(newnode)
        for name, value in data['bars'].items():
            E = value["E"]
            G = value["G"]
            S = value["S"]
            I = value["I"]
            J = value["J"]
            P = np.array([value["P"], value["Px"], value["Px2"]])
            newbar = element(value["id"], self.nodes[value["from"]],self.nodes[value["to"]], E, G, S, I, J, P, name, self.grid)
            self.bars.append(newbar)

    def K(self):
        n = self.n + (self.grid - 1) * (self.n - 1)
        K = np.zeros([2*n,2*n])
        for bar in self.bars:
            i = bar.start.index * bar.n
            K_loc = bar.K_2d()
            K[2*i:2*i+K_loc.shape[0], 2*i:2*i+K_loc.shape[1]] += K_loc
        return K

    def f(self):
        n = self.n + (self.grid - 1) * (self.n - 1)
        f = np.zeros(2*n)
        for bar in self.bars:
            i = bar.start.index * bar.n
            f_loc = bar.f_2d()
            f[2*i:2*i+bar.f_2d().shape[0]-2] += f_loc[0:-2]
        f[-2] += self.bars[-1].f_2d()[-2]
        f[-1] += self.bars[-1].f_2d()[-1]
        return f

    def solve(self):
        n = self.n + (self.grid - 1) * (self.n - 1)
        K = self.K()
        f = self.f()
        u = np.zeros(2*n)
        f_m = f.copy()
        K_m = K.copy()
        for i in range(f.shape[0]):
            j = f.shape[0] - i - 1
            if np.isnan(f[j]):
                f_m = np.delete(f_m, j, 0)
                K_m = np.delete(K_m, j, 0)
                K_m = np.delete(K_m, j, 1)
        u_m = np.linalg.solve(K_m, f_m)
        k = 0
        for i in range(2*n):
            if not np.isnan(f[i]):
                u[i] = u_m[k]
                k += 1
        #f = K@u
        for bar in self.bars:
            i = bar.start.index * bar.n
            j = bar.end.index * bar.n
            u_y = u[2*i]
            u_rz = u[2*i+1]
            u_loc = np.array([0.0,u_y,0.0,0.0,0.0,u_rz])
            bar.start.Desloc(u_loc)
            for k in range(bar.n-1):
                u_y = u[2*(i+k+1)]
                u_rz = u[2*(i+k+1)+1]
                u_loc = np.array([0.0,u_y,0.0,0.0,0.0,u_rz])
                bar.grid[k].Desloc(u_loc)
            u_y = u[2*j]
            u_rz = u[2*j+1]
            u_loc = np.array([0.0,u_y,0.0,0.0,0.0,u_rz])
            bar.end.Desloc(u_loc)
        return

    def show_3d(self):
        out = go.Figure()
        for bar in self.bars:
            local = dict({"x":[],"y":[],"z":[],"tag":[]})
            local["x"].append(bar.start.locate[0])
            local["y"].append(bar.start.locate[1])
            local["z"].append(bar.start.locate[2])
            local["tag"].append(bar.tag)
            for nd in bar.grid:
                local["x"].append(nd.locate[0])
                local["y"].append(nd.locate[1])
                local["z"].append(nd.locate[2])
                local["tag"].append(bar.tag)
            local["x"].append(bar.end.locate[0])
            local["y"].append(bar.end.locate[1])
            local["z"].append(bar.end.locate[2])
            local["tag"].append(bar.tag)
            if self.dimension == 2:
                out.add_scatter(x=local["x"],y=local["y"], marker=dict(size=5), line=dict(width=2), name=bar.tag)
                out.update_layout(legend_title_text = "Barras")
                out.update_xaxes(title_text="$x[m]$")
                out.update_yaxes(title_text="$y[m]$")
            else:
                out.add_scatter3d(x=local["x"],y=local["y"],z=local["z"], marker=dict(size=3), line=dict(width=2), name=bar.tag)
                out.update_layout(legend_title_text = "Barras")
        return out

    def show(self):
        self.solve()
        fig = bk.Figure(title="Structure", sizing_mode="scale_width")
        fig.xaxis.axis_label = 'x [m]'
        fig.yaxis.axis_label = 'y [mm]'
        data = {
            'dot_x': [],
            'dot_y': [],
            'u_x': [],
            'u_y': [],
            'loc_x': [],
            'loc_y': [],
            'M': [],
            'V': []
        }
        data_0 = {
            'dot_x': [],
            'dot_y': [],
            'u_x': [],
            'u_y': [],
            'M': [],
            'V': []
        }
        for dot in self.nodes:
            data_0['dot_x'].append(dot.locate[0])
            data_0['dot_y'].append((dot.locate[1])*1000)
            data_0['u_x'].append(dot.u[0])
            data_0['u_y'].append((dot.u[1])*1000)
            data_0['M'].append(dot.reaction[5])
            data_0['V'].append(dot.reaction[1])
        data['dot_x'].append(self.bars[0].start.locate[0])
        data['dot_y'].append(self.bars[0].start.locate[1])
        data['u_x'].append(self.bars[0].start.u[0]*1000)
        data['u_y'].append(self.bars[0].start.u[1]*1000)
        data['loc_x'].append(self.bars[0].start.locate[0] + self.bars[0].start.u[0])
        data['loc_y'].append((self.bars[0].start.locate[1] + self.bars[0].start.u[1])*1000)
        data['M'].append(self.bars[0].start.reaction[5])
        data['V'].append(self.bars[0].start.reaction[1])
        for bar in self.bars:
            for dot in bar.grid:
                data['dot_x'].append(dot.locate[0])
                data['dot_y'].append(dot.locate[1])
                data['u_x'].append(dot.u[0]*1000)
                data['u_y'].append(dot.u[1]*1000)
                data['loc_x'].append(dot.locate[0] + dot.u[0])
                data['loc_y'].append((dot.locate[1] + dot.u[1])*1000)
                data['M'].append(dot.reaction[5])
                data['V'].append(dot.reaction[1])
            data['dot_x'].append(bar.end.locate[0])
            data['dot_y'].append(bar.end.locate[1])
            data['u_x'].append(bar.end.u[0]*1000)
            data['u_y'].append(bar.end.u[1]*1000)
            data['loc_x'].append(bar.end.locate[0] + bar.end.u[0])
            data['loc_y'].append((bar.end.locate[1] + bar.end.u[1])*1000)
            data['M'].append(bar.end.reaction[5])
            data['V'].append(bar.end.reaction[1])
        source = ColumnDataSource(data=data)
        source_0 = ColumnDataSource(data=data_0)
        hover_tool = HoverTool(
            tooltips=[
                ( 'X',   '@dot_x{0,0.00} m' ),
                ( 'dy', '@u_y{0,0.00} mm'   ),
                ( 'M', '@M{0,0.0} kNm'      ),
                ( 'V', '@V{0,0.0} kN'       ),
            ],
            mode='vline'
        )
        fig.circle(x='dot_x',y='dot_y', source=source_0, size=10, color="navy", alpha=0.5)
        fig.line(x='dot_x',y='dot_y', source=source_0)
        fig.scatter(x='loc_x',y='loc_y', source=source, size=5, color="red", alpha=0.5)
        fig.line(x='loc_x',y='loc_y', source=source,color="gold")
        fig.tools.append(hover_tool)
        return fig

# Exemplo de Arquivo JSON

O exemplo abaixo contém o arquivo JSON da estrutura do Exemplo 1, resolvida analiticamente no tópico anterior. Este estrutura será resolvida novamente abaixo utilizando as classes e métodos apresentados.

In [33]:
#print(json.dumps(readjson("exemplo1.json"), indent=4, sort_keys=True))

# Exemplo 1 - Uma Barra

In [34]:
exemplo1 = Structure(readjson("exemplo1.json"),100)
bk.show(exemplo1.show())

# Exemplo 2 - Três Barras

In [35]:
exemplo2 = Structure(readjson("exemplo2.json"))
bk.show(exemplo2.show())

In [36]:
exemplo3 = Structure(readjson("exemplo3.json"))
bk.show(exemplo3.show())